# Creating the data we are going to be using for the membership cluster
### Here we use the GAIA Eso parameters, GAIA data and Bailer Jones distance estimates

In [1]:
from astropy.io import fits
import os

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
import sklearn.decomposition
import seaborn as sn
import pickle
if("Cluster_analysis" in os.getcwd()):
    os.chdir("../")

from Data_handling.pca_functions import Pca_Fitting
from Data_handling.gaia_query import gaia_cone, bailer_jones_cone, concatenate_gaia_bailer
from Data_handling.vae import Encoder,Decoder, VAE , loss_function, model_train


os.chdir("Cluster_analysis")
os.getcwd()

'/Users/mattocallaghan/GES/Cluster_analysis'

In [2]:
sn.set_style('darkgrid') # darkgrid, white grid, dark, white and ticks
plt.rc('axes', titlesize=18)     # fontsize of the axes title
plt.rc('axes', labelsize=14)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=13)    # fontsize of the tick labels
plt.rc('ytick', labelsize=13)    # fontsize of the tick labels
plt.rc('legend', fontsize=13)    # legend fontsize
plt.rc('font', size=13)         

### Here we are opening th eiDR6 GES file and loading the necessary parameters

In [3]:
os.chdir('../Parameter_files')
par=fits.open('GES_iDR6_WG15_Recommended_with_sflags__mode_normal_091221.fits')
indices=[1,2,3,4,5,6,7,8,15,16,18,19,23,24,25,29,31,32,36,37,38,42,43,45,87,88]
columns=[par[1].header['TTYPE'+str(i)] for i in indices]
data=np.array([par[1].data[x] for x in columns]).transpose()
data=pd.DataFrame(data=data,columns=columns)

# Choose cluster here

In [4]:
OC=data[data['GES_FLD']=='NGC6705'].reset_index()

#opening the files in the cluster 
with open('data_gaia.txt', 'w') as f:
    for item in OC['GAIA_eDR3_ID'].astype(str).values:
        f.write("%s\n" % item)
set(data['GES_FLD'])

{'134_Tau',
 '18_Sco',
 '25_Ori',
 '32_Gem',
 '67_Oph',
 '68_Tau',
 'Arcturus',
 'Assc50',
 'Blanco1',
 'Br20',
 'Br21',
 'Br22',
 'Br25',
 'Br29',
 'Br30',
 'Br31',
 'Br32',
 'Br36',
 'Br39',
 'Br44',
 'Br73',
 'Br75',
 'Br81',
 'Bulge',
 'Cha_I',
 'Col110',
 'Col197',
 'Col261',
 'Corot',
 'Cz24',
 'Cz30',
 'ESO92_05',
 'GES_MW_00_01',
 'GES_MW_01_02',
 'GES_MW_02_03',
 'GES_MW_03_04',
 'GES_MW_04_05',
 'GES_MW_05_06',
 'GES_MW_06_07',
 'GES_MW_07_08',
 'GES_MW_08_09',
 'GES_MW_09_10',
 'GES_MW_10_11',
 'GES_MW_11_12',
 'GES_MW_12_13',
 'GES_MW_13_14',
 'GES_MW_14_15',
 'GES_MW_15_16',
 'GES_MW_16_17',
 'GES_MW_17_18',
 'GES_MW_18_19',
 'GES_MW_19_20',
 'GES_MW_20_21',
 'GES_MW_21_22',
 'GES_MW_22_23',
 'GES_MW_23_24',
 'GJ205',
 'GJ388',
 'GJ436',
 'GJ526',
 'GJ551',
 'GJ581',
 'GJ699',
 'GJ880',
 'HD102200',
 'HD107328',
 'HD109061',
 'HD122563',
 'HD140283',
 'HD148897',
 'HD163758',
 'HD173819',
 'HD190056',
 'HD204587',
 'HD220009',
 'HD22879',
 'HD24937',
 'HD27271',
 'HD283',


### Here we are scraping the GAIA data from servers in a cone of a defined radius,

In [8]:
data

,CNAME,CNAME_iDR6_orig,GAIA_eDR3_ID,DIST_GAIA,GES_FLD,OBJECT,FILENAME,GES_TYPE,RA,DEC,...,FEH,E_FEH,SYS_ERR_FEH,XI,E_XI,VRAD,E_VRAD,VRAD_OFFSET,GAMMA,E_GAMMA
0,00000009-5455467,00000009-5455467,4923847544332011520,0.10800000000000001,GES_MW_00_01,U_2_b_177_13,uvl_00000009-5455467_580.0.fits|uvu_00000009-5...,GE_MW,0.000374999999999994,-54.92963888888799,...,-0.55,0.05,0.09,1.15,0.06,-40.48,0.37,0.3,nan,nan
1,00000010-5727519,00000010-5727519,4919430806123101568,0.0453,GES_MW_00_01,G_2_b_566_225,gir_00000010-5727519_H548.8.fit|gir_00000010-5...,GE_MW,0.00041666666666665965,-57.464416666665706,...,-0.57,0.07,nan,nan,nan,24.66,0.33,nan,nan,nan
2,00000014-6003143,00000014-6003143,4918001166129129856,0.3006,GES_MW_00_01,G_2_r_353_228,gir_00000014-6003143_H548.8.fit|gir_00000014-6...,GE_MW,0.0005833333333333235,-60.05397222222122,...,-0.08,0.07,nan,nan,nan,-21.32,0.11,nan,nan,nan
3,00000064-5739305,00000064-5739305,4919413213937060992,0.0554,GES_MW_00_01,G_2_r_566_162,gir_00000064-5739305_H548.8.fit|gir_00000064-5...,GE_MW,0.002666666666666622,-57.65847222222125,...,-0.83,0.07,nan,nan,nan,-16.93,0.12,nan,nan,nan
4,00000097-5946444,00000097-5946444,4918020201424204288,0.2127,GES_MW_00_01,G_2_e_353_353,gir_00000097-5946444_H548.8.fit|gir_00000097-5...,GE_MW,0.004041666666666598,-59.77899999999899,...,-1.56,0.13,nan,nan,nan,19.05,0.54,nan,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114912,23595755-5502508,23595755-5502508,6497146699679533952,0.0782,GES_MW_23_24,G_2_b_177_163,gir_23595755-5502508_H548.8.fit|gir_23595755-5...,GE_MW,359.98979166666055,-55.04744444444353,...,-1.03,0.11,nan,nan,nan,-34.42,0.59,nan,nan,nan
114913,23595826-2940375,23595826-2940375,2332873338647480576,0.8694,Blanco1,2MASS23595826-29,gir_23595826-2940375_H665.0.fit,GE_CL,359.992749999994,-29.6770833333328,...,0.02,0.05,nan,nan,nan,16.96,0.25,0.09,0.9589,0.0038
114914,23595940-5953322,23595940-5953322,6494228634475449472,0.8894,GES_MW_23_24,U_2_b_353_7,uvl_23595940-5953322_580.0.fits|uvu_23595940-5...,GE_MW,359.997499999994,-59.8922777777768,...,-0.55,0.05,0.09,0.62,0.1,60.88,0.52,0.3,nan,nan
114915,23595962-5455295,23595962-5455295,6497149074797798528,0.1566,GES_MW_23_24,U_2_b_177_6,uvl_23595962-5455295_580.0.fits|uvu_23595962-5...,GE_MW,359.998416666661,-54.924861111110204,...,-0.51,0.04,0.09,0.92,0.08,-38.46,0.37,0.3,nan,nan


In [5]:
ra=OC['RA'].astype(float)
dec=OC['DEC'].astype(float)
radiusd=np.abs(dec.max()-dec.min())
radiusr=np.abs(ra.max()-ra.min())
radius=np.sqrt(radiusr**2+radiusd**2)
center=[ra.mean(),dec.mean()]
radius
selection_region=concatenate_gaia_bailer(center[0],center[1],radius*0.5)

INFO: Query finished. [astroquery.utils.tap.core]
warning, getting rid of very negaitve parallax


In [11]:
selection_region

,solution_id,DESIGNATION,source_id,random_index,ref_epoch,ra,ra_error,dec,dec_error,parallax,...,Source,RA_ICRS,DE_ICRS,rgeo,b_rgeo,B_rgeo,rpgeo,b_rpgeo,B_rpgeo,Flag
0,1636148068921376768,Gaia DR3 4252441450420820352,4252441450420820352,1115778196,2016.0,282.788193,0.090897,-6.547213,0.089533,0.375274,...,4252441450420820352,282.788193,-6.547213,3248.09937,2437.19043,4425.95654,2855.32617,2315.71069,3623.02954,10033
1,1636148068921376768,Gaia DR3 4252441450420820480,4252441450420820480,125166462,2016.0,282.786902,0.223747,-6.548131,0.222232,0.496013,...,4252441450420820480,282.786902,-6.548131,4038.64795,2439.56250,5918.26416,3988.03687,2681.29272,5453.57666,10033
2,1636148068921376768,Gaia DR3 4252441450425473280,4252441450425473280,112301144,2016.0,282.784923,0.185507,-6.546441,0.199230,-0.407128,...,4252441450425473280,282.784923,-6.546441,7774.41602,5147.04150,10613.21190,7394.55908,5955.89648,9597.09766,10033
3,1636148068921376768,Gaia DR3 4252441450445320448,4252441450445320448,813590036,2016.0,282.783954,0.129797,-6.548677,0.130230,0.126955,...,4252441450445320448,282.783954,-6.548677,6052.03955,3953.92188,8247.49414,6454.44092,4869.89355,8585.34473,10033
4,1636148068921376768,Gaia DR3 4252441450445426048,4252441450445426048,1668238841,2016.0,282.788786,0.267401,-6.544377,0.261821,0.270604,...,4252441450445426048,282.788786,-6.544377,7040.88770,4308.13086,9686.02930,NaN,NaN,NaN,10099
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129576,1636148068921376768,Gaia DR3 4254024785860626432,4254024785860626432,1503556053,2016.0,282.699143,0.466856,-6.015281,0.444973,0.290428,...,4254024785860626432,282.699143,-6.015281,6183.77637,4063.00635,9041.96289,5851.25537,3505.85254,7764.89502,10033
129577,1636148068921376768,Gaia DR3 4254024785860628352,4254024785860628352,1609244275,2016.0,282.700588,0.764798,-6.013615,0.982252,-1.867705,...,4254024785860628352,282.700588,-6.013615,4957.20703,2829.87866,7939.29443,5951.43555,3474.25049,9507.98145,10033
129578,1636148068921376768,Gaia DR3 4254024785860646912,4254024785860646912,71372389,2016.0,282.698088,0.500718,-6.012777,0.595257,0.457348,...,4254024785860646912,282.698088,-6.012777,5165.68945,3394.83203,7259.47900,5682.91406,3099.72803,8243.24609,10033
129579,1636148068921376768,Gaia DR3 4254024785870616320,4254024785870616320,1524005956,2016.0,282.702964,1.163964,-6.012223,1.708608,1.579616,...,4254024785870616320,282.702964,-6.012223,6287.07275,3392.10034,9226.73828,9615.29883,7989.99609,11850.23340,10033


In [13]:
OC['GAIA_eDR3_ID']=OC['GAIA_eDR3_ID'].astype(int)
sum(selection_region['GAIA_eDR3_ID']==selection_region['source_id'])-len(selection_region)


0

In [12]:
OC['GAIA_eDR3_ID']=OC['GAIA_eDR3_ID'].astype(int)
selection_region['GAIA_eDR3_ID']=selection_region['source_id']
selection_region

,solution_id,DESIGNATION,source_id,random_index,ref_epoch,ra,ra_error,dec,dec_error,parallax,...,RA_ICRS,DE_ICRS,rgeo,b_rgeo,B_rgeo,rpgeo,b_rpgeo,B_rpgeo,Flag,GAIA_eDR3_ID
0,1636148068921376768,Gaia DR3 4252441450420820352,4252441450420820352,1115778196,2016.0,282.788193,0.090897,-6.547213,0.089533,0.375274,...,282.788193,-6.547213,3248.09937,2437.19043,4425.95654,2855.32617,2315.71069,3623.02954,10033,4252441450420820352
1,1636148068921376768,Gaia DR3 4252441450420820480,4252441450420820480,125166462,2016.0,282.786902,0.223747,-6.548131,0.222232,0.496013,...,282.786902,-6.548131,4038.64795,2439.56250,5918.26416,3988.03687,2681.29272,5453.57666,10033,4252441450420820480
2,1636148068921376768,Gaia DR3 4252441450425473280,4252441450425473280,112301144,2016.0,282.784923,0.185507,-6.546441,0.199230,-0.407128,...,282.784923,-6.546441,7774.41602,5147.04150,10613.21190,7394.55908,5955.89648,9597.09766,10033,4252441450425473280
3,1636148068921376768,Gaia DR3 4252441450445320448,4252441450445320448,813590036,2016.0,282.783954,0.129797,-6.548677,0.130230,0.126955,...,282.783954,-6.548677,6052.03955,3953.92188,8247.49414,6454.44092,4869.89355,8585.34473,10033,4252441450445320448
4,1636148068921376768,Gaia DR3 4252441450445426048,4252441450445426048,1668238841,2016.0,282.788786,0.267401,-6.544377,0.261821,0.270604,...,282.788786,-6.544377,7040.88770,4308.13086,9686.02930,NaN,NaN,NaN,10099,4252441450445426048
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129576,1636148068921376768,Gaia DR3 4254024785860626432,4254024785860626432,1503556053,2016.0,282.699143,0.466856,-6.015281,0.444973,0.290428,...,282.699143,-6.015281,6183.77637,4063.00635,9041.96289,5851.25537,3505.85254,7764.89502,10033,4254024785860626432
129577,1636148068921376768,Gaia DR3 4254024785860628352,4254024785860628352,1609244275,2016.0,282.700588,0.764798,-6.013615,0.982252,-1.867705,...,282.700588,-6.013615,4957.20703,2829.87866,7939.29443,5951.43555,3474.25049,9507.98145,10033,4254024785860628352
129578,1636148068921376768,Gaia DR3 4254024785860646912,4254024785860646912,71372389,2016.0,282.698088,0.500718,-6.012777,0.595257,0.457348,...,282.698088,-6.012777,5165.68945,3394.83203,7259.47900,5682.91406,3099.72803,8243.24609,10033,4254024785860646912
129579,1636148068921376768,Gaia DR3 4254024785870616320,4254024785870616320,1524005956,2016.0,282.702964,1.163964,-6.012223,1.708608,1.579616,...,282.702964,-6.012223,6287.07275,3392.10034,9226.73828,9615.29883,7989.99609,11850.23340,10033,4254024785870616320


In [14]:
gaia_df_gc=selection_region[selection_region['GAIA_eDR3_ID'].isin(OC['GAIA_eDR3_ID'].astype(int).values)]

In [15]:
gaia_not_eso=selection_region[~selection_region['GAIA_eDR3_ID'].isin(OC['GAIA_eDR3_ID'].astype(int).values)]

In [16]:
gaia_eso=pd.DataFrame.merge(OC, gaia_df_gc,on='GAIA_eDR3_ID')

# Save now under any name

In [17]:
os.chdir('../cluster_files')

In [18]:
gaia_eso.to_csv('gaia_eso.csv')
gaia_not_eso.to_csv('gaia_not_eso.csv')

In [19]:
pd.read_csv('gaia_eso.csv')

,Unnamed: 0,index,CNAME,CNAME_iDR6_orig,GAIA_eDR3_ID,DIST_GAIA,GES_FLD,OBJECT,FILENAME,GES_TYPE,...,Source,RA_ICRS,DE_ICRS,rgeo,b_rgeo,B_rgeo,rpgeo,b_rpgeo,B_rpgeo,Flag
0,0,95849,18501559-0615288,18501559-0615288,4252510277385184512,0.3335,NGC6705,18501559-0615287,gir_18501559-0615288_H665.0.fit,GE_CL,...,4252510277385184512,282.565031,-6.258065,1942.98157,1618.71545,2489.67163,1923.29565,1666.32056,2345.46167,10033
1,1,95859,18502001-0611319,18502001-0611319,4252516629530920192,0.2550,NGC6705,18502001-0611319,NaN,GE_CL,...,4252516629530920192,282.583418,-6.192254,6141.48389,3163.27856,10115.85840,3446.87402,2693.09326,5311.75977,10033
2,2,95860,18502037-0622456,18502037-0622456,4252495055982423040,0.2510,NGC6705,18502037-0622455,gir_18502037-0622456_H665.0.fit,GE_CL,...,4252495055982423040,282.584937,-6.379368,7317.13623,4718.66797,10304.85840,5302.03076,4131.42090,6951.20215,10033
3,3,95865,18502227-0619460,18502227-0619460,4252508696836953088,0.3968,NGC6705,18502227-0619460,gir_18502227-0619460_H665.0.fit,GE_CL,...,4252508696836953088,282.592885,-6.329504,2671.46338,2193.52368,3324.61035,2677.84912,2291.58057,3349.83423,10033
4,4,95868,18502321-0614292,18502321-0614292,4252510380464366080,0.0700,NGC6705,18502321-0614292,gir_18502321-0614292_H665.0.fit,GE_CL,...,4252510380464366080,282.596724,-6.241452,1887.64343,1726.50842,2108.31055,1788.85913,1661.42944,1947.09448,10033
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1041,1041,97639,18514208-0618480,18514208-0618480,4252499969462698624,0.1117,NGC6705,18514208-0618480,gir_18514208-0618480_H665.0.fit,GE_CL,...,4252499969462698624,282.925362,-6.313343,1479.17065,1300.45764,1680.05493,1470.31299,1358.24792,1639.43884,10033
1042,1042,97661,18514395-0614538,18514395-0614538,4252501618730217344,0.1863,NGC6705,18514395-0614538,gir_18514395-0614538_H665.0.fit,GE_CL,...,4252501618730217344,282.933178,-6.248270,1865.90308,1691.89001,2057.87891,1799.89722,1642.38660,2016.46265,10033
1043,1043,97675,18514499-0616300,18514499-0616300,4252500691017240192,0.2774,NGC6705,18514499-0616300,NaN,GE_CL,...,4252500691017240192,282.937478,-6.275078,2512.78027,2140.38989,3003.52466,2258.85547,1983.30444,2657.37671,10033
1044,1044,97681,18514666-0616457,18514666-0616457,4252500617885643520,0.1877,NGC6705,18514666-0616457,gir_18514666-0616457_H665.0.fit,GE_CL,...,4252500617885643520,282.944467,-6.279355,4087.95532,2831.92578,6757.66211,3010.33057,2544.00537,4031.78394,10033
